# Develop minERR

In [1]:
from Bio import SeqIO, SeqRecord
import gzip
from mineer.mineer import Read

In [2]:
file = 'fastqs/SRR9660307_1.fastq.gz'
with gzip.open(file, 'rt') as fh:
    reads = [Read(r) for r in SeqIO.parse(fh, 'fastq')]

In [3]:
%%time
for r in reads[:500]:
    r.trim()

CPU times: user 859 ms, sys: 10.6 ms, total: 870 ms
Wall time: 871 ms
